In [ ]:
from google.colab import files
uploaded = files.upload()  # Opens file upload dialog

Saving final_preprocessed_data.csv to final_preprocessed_data.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
df = pd.read_csv("final_preprocessed_data.csv")

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-0.756450,-0.080555,-0.887220,0.207242,2.416118,2.586903,1.521106,-0.975096,-1.130560,1.595359,...,-1.224920,-0.499905,-0.460336,-1.019537,-0.137944,0.620807,0.909807,1.557280,2.552950,sad
1,1.599247,1.855606,-2.563416,1.001680,1.497387,-1.669671,-1.141146,-1.209192,-1.529407,-0.644377,...,-0.757056,0.529546,0.797697,0.405281,1.235196,0.961448,1.041838,1.742813,1.617229,angry
2,-4.684983,1.562893,0.781379,1.804534,2.364849,1.227898,0.546017,0.312357,-0.082968,1.300100,...,0.954061,0.140788,-0.213432,-0.321112,0.912375,1.762079,1.173572,0.538463,0.987126,disgust
3,-0.616899,-0.080653,-0.342846,1.034165,0.875514,-0.683095,0.833404,0.253707,-1.966081,-0.251808,...,-0.532205,0.429465,0.617555,-0.080381,0.770216,1.318908,1.024836,1.872452,2.978115,disgust
4,0.015079,1.005145,-0.827219,1.819852,2.169953,0.607013,0.102312,-1.716277,-1.121451,1.046999,...,-0.176075,-0.316824,-0.423309,0.031363,0.547154,0.522942,1.415464,2.187452,2.351905,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7917,0.450498,-0.905687,-0.547986,1.160502,-0.482736,-1.021221,0.925477,0.381166,-0.221137,0.459160,...,-0.008017,0.455512,0.562067,-0.093431,-0.474984,-0.561987,0.099760,0.309969,-0.037905,angry
7918,0.480462,1.190142,-0.603195,-0.635210,-0.580023,-0.771256,-0.618595,-0.235845,0.358206,-0.582249,...,0.168656,-0.528430,0.067805,0.648247,1.874564,0.760622,-1.103290,-0.891329,-0.188172,angry
7919,-0.628838,-0.037181,-0.677894,0.150460,0.580951,-0.144128,-0.308272,0.510370,-1.086582,-1.863288,...,-1.529005,-2.056011,-1.680400,-0.495962,1.251208,1.560294,2.245954,-0.227367,-1.463024,unknown
7920,-1.277788,-1.362675,0.956572,1.013365,1.424563,1.081627,-0.880253,1.644329,1.483518,0.215023,...,-1.349755,-1.164908,-1.419604,-0.354005,-1.394434,-1.791711,-1.147715,-1.845789,0.255492,disgust


In [ ]:
# Remove "unknown" category
df_filtered = df[df["emotion"] != "unknown"]

# Verify new class distribution
print(df_filtered["emotion"].value_counts())

emotion
sad          1331
angry        1331
disgust      1331
happy        1331
fearful      1331
neutral       120
surprised      60
Name: count, dtype: int64


In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Extract features & labels after removing "unknown"
X = df_filtered.iloc[:, :-1].values
y = df_filtered["emotion"].values

# Apply oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Convert back to DataFrame
df_balanced = pd.DataFrame(X_resampled, columns=df_filtered.columns[:-1])
df_balanced["emotion"] = y_resampled

# Verify class distribution after oversampling
print(df_balanced["emotion"].value_counts())

emotion
sad          1331
angry        1331
disgust      1331
happy        1331
surprised    1331
neutral      1331
fearful      1331
Name: count, dtype: int64


In [ ]:
X = df_balanced.drop("emotion", axis=1).values
y = df_balanced["emotion"].values

In [ ]:
y

array(['sad', 'angry', 'disgust', ..., 'surprised', 'surprised',
       'surprised'], dtype=object)

In [ ]:
X

array([[-0.75644964, -0.08055476, -0.8872204 , ...,  0.9098071 ,
         1.5572802 ,  2.5529501 ],
       [ 1.5992473 ,  1.8556056 , -2.5634162 , ...,  1.0418382 ,
         1.7428125 ,  1.6172293 ],
       [-4.684983  ,  1.5628929 ,  0.78137934, ...,  1.1735723 ,
         0.53846294,  0.9871258 ],
       ...,
       [ 0.3013906 ,  0.12918194, -1.0484655 , ...,  3.5296624 ,
         2.9592083 ,  2.5735521 ],
       [-0.01860063, -0.2721646 , -1.7854614 , ...,  2.0502229 ,
         3.0079336 ,  2.9565325 ],
       [-0.08681267, -0.2962078 , -1.9706056 , ...,  1.6139055 ,
         1.8939035 ,  1.4280283 ]])

In [ ]:
df = df_balanced[df_balanced["emotion"].str.lower() != "unknown"]

In [ ]:
X = df.drop("emotion", axis=1).values
y = df["emotion"].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Build CNN model
model = Sequential()
model.add(Conv1D(128, 5, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(256, 5, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_onehot.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[reduce_lr])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 74ms/step - accuracy: 0.4647 - loss: 1.7279 - val_accuracy: 0.2661 - val_loss: 3.1264 - learning_rate: 0.0010
Epoch 2/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 78ms/step - accuracy: 0.5876 - loss: 1.0366 - val_accuracy: 0.2639 - val_loss: 3.2651 - learning_rate: 0.0010
Epoch 3/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step - accuracy: 0.6081 - loss: 0.9687 - val_accuracy: 0.5102 - val_loss: 1.5567 - learning_rate: 0.0010
Epoch 4/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.6284 - loss: 0.9240 - val_accuracy: 0.6239 - val_loss: 1.0256 - learning_rate: 0.0010
Epoch 5/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.6547 - loss: 0.8636 - val_accuracy: 0.6202 - val_loss: 0.9508 - learning_rate: 0.0010
Epoch 6/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.6655 - loss: 0.8365 - val_accuracy: 0.6502 - val_loss: 0.9124 - learning_rate: 0.0010
Epoch 7/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.6703 - loss

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6502 - loss: 0.9271

Test Accuracy: 0.6540


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Re-encode labels
label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(df["emotion"])

# Use SMOTE to oversample
X = df.drop("emotion", axis=1).values
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_labels)

# One-hot encode labels after SMOTE
y_onehot = to_categorical(y_resampled)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
)

# Reshape for Conv1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Build CNN model
model = Sequential()
model.add(Conv1D(128, 5, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(256, 5, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_onehot.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[reduce_lr])

Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - accuracy: 0.4607 - loss: 1.7127 - val_accuracy: 0.1427 - val_loss: 6.1129 - learning_rate: 0.0010
Epoch 2/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - accuracy: 0.5954 - loss: 1.0094 - val_accuracy: 0.1481 - val_loss: 6.8109 - learning_rate: 0.0010
Epoch 3/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.6143 - loss: 0.9458 - val_accuracy: 0.4142 - val_loss: 2.9283 - learning_rate: 0.0010
Epoch 4/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.6301 - loss: 0.8987 - val_accuracy: 0.5992 - val_loss: 1.1276 - learning_rate: 0.0010
Epoch 5/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.6485 - loss: 0.8476 - val_accuracy: 0.6384 - val_loss: 0.9622 - learning_rate: 0.0010
Epoch 6/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - accuracy: 0.6736 - loss: 0.8012 - val_accuracy: 0.6411 - val_loss: 0.9519 - learning_rate: 0.0010
Epoch 7/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.6900 - loss

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6624 - loss: 1.0048

Test Accuracy: 0.6615
